# 步骤二：（仅alice和bob）数据加密

数据在进入TEE计算之前，需要对数据进行加密。

TrustFlow提供了[CLI工具](https://github.com/secretflow/capsule-manager-sdk)来协助您完成相关操作，CLI工具提供了数据加密、授权和导出等功能。

如果您想了解更多数据加密的原理，可以参见[TrustFlow原理](../architecture/principle.md)。

## 第一步：下载CLI工具

> 我们建议您在trustflow通用release镜像中安装和使用cli工具，在通用镜像中我们预先安装了conda和Intel Dcap库。进入镜像后先创建python3.10环境再进行sdk的安装，请确认安装的是最新版本的capsule-manager-sdk。具体命令如下：

```bash
docker run -it --name capsule-manager-sdk --network=host secretflow/trustflow-release-ubuntu22.04:latest bash

conda create -n capsule-manager-sdk python=3.10 -y
conda activate capsule-manager-sdk
pip install --upgrade capsule-manager-sdk
```

> 如果您想在您的系统上直接安装使用cli工具，请先自行安装conda并创建python3.10环境，并且根据你不同的操作系统版本参照[Intel SGX SW Installation Guide](https://download.01.org/intel-sgx/latest/linux-latest/docs/Intel_SGX_SW_Installation_Guide_for_Linux.pdf)安装好Dcap库。然后再执行
```bash
conda create -n capsule-manager-sdk python=3.10 -y
conda activate capsule-manager-sdk
pip install --upgrade capsule-manager-sdk
```

CLI工具详细的功能和使用方式详细参见[capsule-manager-sdk](https://github.com/secretflow/capsule-manager-sdk)，这里我们简短说明一下工具提供的命令。

- cms_config: 对工具的配置文件进行设置，我们提供了[模板配置文件](https://github.com/secretflow/capsule-manager-sdk/blob/main/python/cli/cli-template.yaml)，您可以复制该配置文件到您自己的路径下，然后通过--config-file指定配置文件的路径。
- cms: 访问CapsuleManager服务。
- cms_util: 提供诸如加密、解密等功能。

## 第二步：加密数据

这里我们以开源数据集[breast cancer](https://archive.ics.uci.edu/dataset/17/breast+cancer+wisconsin+diagnostic)为例，breast cancer是由加利福尼亚大学尔湾分校（UCI）提供的乳腺癌诊断数据，该数据集一共包含569个样本，每个样本拥有一个ID和10个特征，是一个典型的二分类数据集。

假设breast cancer是由alice和bob两个机构分别持有，为了方便演示，我们已经提前将breast cancer数据集拆分成了两部分，alice拥有前面5个特征，bob拥有后5个特征。拆分后的数据集可以点击链接进行下载，alice只需下载alice.csv，bob只需下载bob.csv。

[alice.csv](https://secretflow-data.oss-cn-shanghai.aliyuncs.com/datasets/breast_cancer/alice.csv)
[bob.csv](https://secretflow-data.oss-cn-shanghai.aliyuncs.com/datasets/breast_cancer/bob.csv)

1. alice生成数据密钥，用于加密数据。（该密钥请妥善保存，请勿泄露给他人）

执行后将会输出一个BASE64编码后的随机16字节的数据密钥，您也可以使用其他方式自行生成数据密钥。

```bash
cms_util generate-data-key-b64
```
2. alice加密数据文件，记得替换数据密钥为上一步得到的密钥，加密后的文件保存为alice.csv.enc
```bash
cms_util encrypt-file --source-file alice.csv --dest-file alice.csv.enc --data-key-b64 数据密钥
```
3. bob同样的执行1和2步骤，记得bob要加密的文件是bob.csv，加密后得到的文件为bob.csv.enc

## 第三步：上传数据密钥

数据加密后，需要把数据密钥安全的传输给CapsuleManager。CLI工具提供了对应能力。

CLI工具需要根据配置文件进行工作。我们提供了[模板配置文件](https://github.com/secretflow/capsule-manager-sdk/blob/main/python/cli/cli-template.yaml)，您可以下载到本地，后续基于这个模板进行配置。

接下来您需要对配置文件进行修改以完成数据密钥上传。

### 选项一：仿真模式

1. alice复制cli-template.yaml命名为alice.yaml，并对alice.yaml进行配置，主要配置CapsuleManager的地址、TEE的平台类型（以及TLS证书，如果开启了tls模式，一般不需要开启）。

相关参数说明如下：
- `--host`填写CapsuleManager的服务地址。
- `--tee-plat`填写tee平台类型，目前支持sim/sgx/tdx/csv，仿真模式下填写sim。
- `--root-ca-file`、`--private-key-file`、`--cert-chain-file`只有在CapsuleManager开启mTLS功能后才需要填写，一般不需要开启该功能，因此可以跳过这三个配置。
**请注意，这里的三个证书为tls证书，不要与后续步骤中所说的机构证书如alice.crt混淆。不开启mTls功能可以不配置tls证书，但是机构证书必须配置。**
**如果您遗漏了机构证书产生的过程，可以回顾[机构证书](./step0.ipynb#机构证书)章节为机构产生自签证书。**

```bash
cp cli-template.yaml alice.yaml
cms_config --config-file alice.yaml init --host {CapsuleManager的服务地址} --tee-plat sim
```

（cms_config命令只是辅助您对配置文件进行设置。您也可以不使用cms_config，手动修改yaml文件进行配置）

2. alice获取自身的机构ID，机构ID是一个BASE32编码的客户端公钥的哈希值，可以根据机构证书生成

产生机构ID的命令如下：
```bash
cms_util generate-party-id --cert-file alice.crt
```

3. alice配置party-id、证书、私钥、加密算法等

相关参数说明如下：
- `party-id`即为上一步中产生的机构ID。
- `--schema`填写为RSA。
- `--cert-pems-file`填写机构证书路径。如果您的证书是来自[机构证书](./step0.ipynb#机构证书)中自签证书，则只需要填写单个文件；如果您的证书来自ca机构签发，那么将证书文件按照[cert, mid_ca_cert, root_ca_cert]的顺序填写。

```bash
cms_config --config-file alice.yaml common --party-id xxx --scheme RSA --cert-pems-file alice.crt --private-key-file alice.key
```

（cms_config命令只是辅助您对配置文件进行设置。您也可以不使用cms_config，手动修改yaml文件进行配置）

4. alice配置上传密钥信息，请在alice.yaml文件中手动配置

`data_keys`是一个列表，因为我们支持同时上传多个密钥，每个密钥包含以下信息：

- `resource_uri`：我们给加密数据取一个名称，在后面授权的时候会用到。比如，我们把这份数据叫做`breast_cancer_alice`。这个名称后续会用到。
- `data_key_b64`：填写上一步得到的数据加密密钥。

```yaml
register_data_keys:
  data_keys:
    - 
      # (required) str
      resource_uri: breast_cancer_alice
      # (required) str
      data_key_b64: xxxx
```

5. alice上传密钥

```bash
cms --config-file alice.yaml register-data-keys
```

6. bob类似地执行前面1~5步。复制cli-template.yaml命名为bob.yaml，并对bob.yaml进行配置。假设bob的数据名字叫做`breast_cancer_bob`,即第4步中的`resource_uri`填写`breast_cancer_bob`。

完成配置后bob也上传自己的密钥

```bash
cms --config-file bob.yaml register-data-keys
```

### 选项二：SGX模式

与仿真模式相比，SGX模式下，CLI工具会对CapsuleManager进行[远程认证](../architecture/tee/sgx.md#远程认证)，以确保CapsuleManager的代码和部署符合预期。

远程认证中我们可以对CapsuleManager的度量值进行约束，也就是设置预期的mrenclave（关于mrenclave的解释可以阅读[资料](../architecture/tee/sgx.md#enclave身份标识---mrenclave-和-mrsigner)），[步骤一](./step1.ipynb)的选项二描述了如何获得CapsuleManager的mrencalve。

您可以参照仿真模式下的说明，差别点仅在第1步需要做以下修改：

- `--tee-plat`填写tee平台类型，目前支持sim/sgx/tdx/csv，SGX模式下填写sgx。
- `--mr-ta`填写CapsuleManager的mrenclave值。

```bash
cms_config --config-file alice.yaml init --host {CapsuleManager的服务地址} --tee-plat sgx --mr-ta xxxx
```

剩余步骤与仿真模式一致。


### 选项三：TDX模式

TDX模式下，CLI工具会对CapsuleManager所在的TD VM进行远程认证。

您可以参照仿真模式下的说明，差别点仅在第1步需要做以下修改：

- `--tee-plat`填写tee平台类型，目前支持sim/sgx/tdx/csv，TDX模式下填写tdx。

由于TD VM的度量值暂无简易工具可以获取，TDX模式下我们暂时不对CapsuleManager的度量值进行限制，您可以保留远程认证报告用于后验。

```bash
cms_config --config-file alice.yaml init --host {CapsuleManager的服务地址} --tee-plat tdx
```

剩余步骤与仿真模式一致。

### 选项四：CSV模式

CSV模式下，CLI工具会对CapsuleManager所在的CSV VM进行远程认证。

您可以参照仿真模式下的说明，差别点仅在第1步需要做以下修改：

- `--tee-plat`填写tee平台类型，目前支持sim/sgx/tdx/csv，CSV模式下填写csv。

由CSV VM的度量值暂无简易工具可以获取，CSV模式下我们暂时不对CapsuleManager的度量值进行限制，您可以保留远程认证报告用于后验。

```bash
cms_config --config-file alice.yaml init --host {CapsuleManager的服务地址} --tee-plat csv
```

剩余步骤与仿真模式一致。